<a href="https://colab.research.google.com/github/kikiru328/enterprise_analysis/blob/main/B_clf_ratio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Introduction  
>대분류별 구매량 비율

In [1]:
import os, sys
from google.colab import drive
drive.mount('/content/drive')
my_path = '/content/notebooks'
os.symlink('/content/drive/My Drive/Colab Notebooks', my_path)
sys.path.insert(0,my_path)

Mounted at /content/drive


In [2]:
#  Directory 변경 (모듈이용)
%pwd
%cd '../content/notebooks'

# module import
## Data Load
from pickle5 import pickle 

## Basic module
import pandas as pd
import numpy as np
import datetime as dt
import seaborn as sns
import matplotlib.pyplot as plt
plt.rc('font', family='NanumBarunGothic')


/content/drive/My Drive/Colab Notebooks


In [3]:
with open('/content/drive/MyDrive/공유문서/1차 프로젝트/5조/data/EDA/기존_purprd_cust.pkl','rb') as file:
  df = pickle.load(file)


In [4]:
df

,성별,연령대,거주지역,제휴사,멤버십가입개수,영수증번호,고객번호,점포코드,구매일자,구매년도,구매월,구매시간,대분류명,중분류명,소분류명,구매금액
0,F,50대,경기 김포시,A,0,2757986,15148,10,2014-09-20,2014,9,20,가구/인테리어,일용잡화,위생세제,1000
1,F,50대,경기 김포시,A,0,2757986,15148,10,2014-09-20,2014,9,20,가공식품,가공식품,일반가공식품,4100
2,F,60세이상,경기 김포시,A,0,2757987,400,10,2014-09-20,2014,9,12,신선식품,농산물,채소,5000
3,F,60세이상,경기 김포시,A,0,2757987,400,10,2014-09-20,2014,9,12,가공식품,젓갈/반찬,벌크김치,10200
4,F,50대,경기 김포시,A,0,2757988,2043,10,2014-09-20,2014,9,13,가구/인테리어,일용잡화,위생세제,3950
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28593025,M,30대,경기 김포시,D,1,253839,15835,40,2015-10-26,2015,10,13,가공식품,음료,일반차음료,3000
28593026,M,30대,경기 김포시,D,1,253839,15835,40,2015-10-26,2015,10,13,가공식품,음료,커피음료,1300
28593027,M,50대,울산 동구,D,0,253840,2070,40,2015-10-26,2015,10,16,일상용품,헤어케어,샴푸,59700
28593028,M,30대,경기 김포시,D,1,253841,15835,40,2015-10-26,2015,10,23,가공식품,과자,하드캔디,2000


In [90]:
df_1 = df[(df['구매년도']==2014)&(df['구매월']>=1) & (df['구매월']<=6)] 
df_2 = df[(df['구매년도']==2014)&(df['구매월']>=7) & (df['구매월']<=12)]
df_3 = df[(df['구매년도']==2015)&(df['구매월']>=1) & (df['구매월']<=6)]
df_4 = df[(df['구매년도']==2015)&(df['구매월']>=7) & (df['구매월']<=12)]

In [91]:
t = pd.get_dummies(df_1['대분류명'])
print(t.shape)
df_1.shape

(6704092, 11)


(6704092, 16)

In [95]:
def ratio(df):
  test = df[['고객번호','대분류명']]
  test = test.set_index('고객번호')
  A = pd.get_dummies(test['대분류명']).reset_index()
  A = A.groupby('고객번호').sum().astype(int)
  Li = list(A.columns)
  A['total'] = A.sum(axis=1).astype(int)
  for i in Li:
    A[f"{i}_r"] = A[i] / A['total']
  B = A.drop(['가공식품', '가구/인테리어', '교육/문화용품', '기타', '디지털/가전', '명품', '신선식품', '의류',
    '일상용품', '전문스포츠/레저', '패션잡화','total'],axis=1)
  return B

In [109]:
H1_r = ratio(df_1)
H2_r = ratio(df_2)
H3_r = ratio(df_3)
H4_r = ratio(df_4)

In [110]:
H1_r

,가공식품_r,가구/인테리어_r,교육/문화용품_r,기타_r,디지털/가전_r,명품_r,신선식품_r,의류_r,일상용품_r,전문스포츠/레저_r,패션잡화_r
고객번호,,,,,,,,,,,
1,0.277311,0.037815,0.0,0.004202,0.000000,0.000000,0.306723,0.155462,0.100840,0.046218,0.071429
2,0.141593,0.091445,0.0,0.000000,0.011799,0.000000,0.566372,0.038348,0.070796,0.032448,0.047198
3,0.681034,0.000000,0.0,0.000000,0.000000,0.000000,0.186782,0.002874,0.129310,0.000000,0.000000
4,0.333333,0.025926,0.0,0.000000,0.000000,0.000000,0.433333,0.103704,0.062963,0.011111,0.029630
5,0.320955,0.074271,0.0,0.000000,0.002653,0.002653,0.551724,0.013263,0.015915,0.002653,0.015915
...,...,...,...,...,...,...,...,...,...,...,...
19356,0.617021,0.000000,0.0,0.000000,0.000000,0.000000,0.340426,0.000000,0.042553,0.000000,0.000000
19357,0.583333,0.000000,0.0,0.000000,0.000000,0.000000,0.166667,0.250000,0.000000,0.000000,0.000000
19359,0.647059,0.000000,0.0,0.000000,0.000000,0.000000,0.294118,0.000000,0.058824,0.000000,0.000000


In [98]:
def reName(a,df):
    cols = list(df.columns)
    for i in cols:
      df[f'{a}_{i}'] = df[i]
    A = df.drop(cols,axis=1)
    return A

H1 = reName('H1',H1_r)
H2 = reName('H2',H2_r)
H3 = reName('H3',H3_r)
H4 = reName('H4',H4_r)

In [99]:
H1

,H1_가공식품_r,H1_가구/인테리어_r,H1_교육/문화용품_r,H1_기타_r,H1_디지털/가전_r,H1_명품_r,H1_신선식품_r,H1_의류_r,H1_일상용품_r,H1_전문스포츠/레저_r,H1_패션잡화_r
고객번호,,,,,,,,,,,
1,0.277311,0.037815,0.0,0.004202,0.000000,0.000000,0.306723,0.155462,0.100840,0.046218,0.071429
2,0.141593,0.091445,0.0,0.000000,0.011799,0.000000,0.566372,0.038348,0.070796,0.032448,0.047198
3,0.681034,0.000000,0.0,0.000000,0.000000,0.000000,0.186782,0.002874,0.129310,0.000000,0.000000
4,0.333333,0.025926,0.0,0.000000,0.000000,0.000000,0.433333,0.103704,0.062963,0.011111,0.029630
5,0.320955,0.074271,0.0,0.000000,0.002653,0.002653,0.551724,0.013263,0.015915,0.002653,0.015915
...,...,...,...,...,...,...,...,...,...,...,...
19356,0.617021,0.000000,0.0,0.000000,0.000000,0.000000,0.340426,0.000000,0.042553,0.000000,0.000000
19357,0.583333,0.000000,0.0,0.000000,0.000000,0.000000,0.166667,0.250000,0.000000,0.000000,0.000000
19359,0.647059,0.000000,0.0,0.000000,0.000000,0.000000,0.294118,0.000000,0.058824,0.000000,0.000000


In [100]:
def train_df(a,b,c):
    A = pd.concat([a,b],axis=1)
    train = pd.concat([A,c],axis=1)
    return train

train = train_df(H1,H2,H3)

In [101]:
train

,H1_가공식품_r,H1_가구/인테리어_r,H1_교육/문화용품_r,H1_기타_r,H1_디지털/가전_r,H1_명품_r,H1_신선식품_r,H1_의류_r,H1_일상용품_r,H1_전문스포츠/레저_r,H1_패션잡화_r,H2_가공식품_r,H2_가구/인테리어_r,H2_교육/문화용품_r,H2_기타_r,H2_디지털/가전_r,H2_명품_r,H2_신선식품_r,H2_의류_r,H2_일상용품_r,H2_전문스포츠/레저_r,H2_패션잡화_r,H3_가공식품_r,H3_가구/인테리어_r,H3_교육/문화용품_r,H3_기타_r,H3_디지털/가전_r,H3_명품_r,H3_신선식품_r,H3_의류_r,H3_일상용품_r,H3_전문스포츠/레저_r,H3_패션잡화_r
고객번호,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0.277311,0.037815,0.0,0.004202,0.000000,0.000000,0.306723,0.155462,0.100840,0.046218,0.071429,0.308989,0.101124,0.000000,0.002809,0.02809,0.000000,0.300562,0.132022,0.095506,0.000000,0.030899,0.333333,0.060897,0.000000,0.000000,0.003205,0.003205,0.355769,0.144231,0.041667,0.016026,0.041667
2,0.141593,0.091445,0.0,0.000000,0.011799,0.000000,0.566372,0.038348,0.070796,0.032448,0.047198,0.236162,0.081181,0.007380,0.000000,0.00738,0.003690,0.461255,0.081181,0.066421,0.029520,0.025830,0.149425,0.084291,0.000000,0.000000,0.011494,0.015326,0.379310,0.076628,0.203065,0.026820,0.053640
3,0.681034,0.000000,0.0,0.000000,0.000000,0.000000,0.186782,0.002874,0.129310,0.000000,0.000000,0.672634,0.000000,0.000000,0.000000,0.00000,0.000000,0.173913,0.000000,0.153453,0.000000,0.000000,0.704545,0.000000,0.000000,0.000000,0.000000,0.000000,0.159091,0.000000,0.136364,0.000000,0.000000
4,0.333333,0.025926,0.0,0.000000,0.000000,0.000000,0.433333,0.103704,0.062963,0.011111,0.029630,0.319767,0.069767,0.000000,0.000000,0.00000,0.000000,0.331395,0.104651,0.110465,0.029070,0.034884,0.349810,0.034221,0.000000,0.000000,0.000000,0.000000,0.479087,0.060837,0.053232,0.007605,0.015209
5,0.320955,0.074271,0.0,0.000000,0.002653,0.002653,0.551724,0.013263,0.015915,0.002653,0.015915,0.292308,0.038462,0.000000,0.000000,0.00000,0.000000,0.615385,0.007692,0.023077,0.007692,0.015385,0.382114,0.008130,0.000000,0.000000,0.000000,0.000000,0.569106,0.008130,0.032520,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19356,0.617021,0.000000,0.0,0.000000,0.000000,0.000000,0.340426,0.000000,0.042553,0.000000,0.000000,0.706037,0.002625,0.000000,0.000000,0.00000,0.000000,0.217848,0.000000,0.073491,0.000000,0.000000,0.654378,0.000000,0.000000,0.000000,0.000000,0.000000,0.218894,0.000000,0.126728,0.000000,0.000000
19357,0.583333,0.000000,0.0,0.000000,0.000000,0.000000,0.166667,0.250000,0.000000,0.000000,0.000000,0.710588,0.002353,0.016471,0.016471,0.00000,0.002353,0.134118,0.025882,0.075294,0.009412,0.007059,0.684327,0.008830,0.024283,0.024283,0.000000,0.002208,0.174393,0.011038,0.059603,0.002208,0.008830
19359,0.647059,0.000000,0.0,0.000000,0.000000,0.000000,0.294118,0.000000,0.058824,0.000000,0.000000,0.713483,0.000000,0.000000,0.000000,0.00000,0.000000,0.232210,0.000000,0.054307,0.000000,0.000000,0.779126,0.000000,0.000000,0.000000,0.000000,0.000000,0.179612,0.000000,0.041262,0.000000,0.000000


In [116]:
trainlist = ['H1','H2','H3']
c_li = list(H1_r.columns)


trainList = []
for col in c_li:
  for h in trainlist:
    trainList.append(f'{h}_{col}')

train = train[trainList]

train

,H1_가공식품_r,H2_가공식품_r,H3_가공식품_r,H1_가구/인테리어_r,H2_가구/인테리어_r,H3_가구/인테리어_r,H1_교육/문화용품_r,H2_교육/문화용품_r,H3_교육/문화용품_r,H1_기타_r,H2_기타_r,H3_기타_r,H1_디지털/가전_r,H2_디지털/가전_r,H3_디지털/가전_r,H1_명품_r,H2_명품_r,H3_명품_r,H1_신선식품_r,H2_신선식품_r,H3_신선식품_r,H1_의류_r,H2_의류_r,H3_의류_r,H1_일상용품_r,H2_일상용품_r,H3_일상용품_r,H1_전문스포츠/레저_r,H2_전문스포츠/레저_r,H3_전문스포츠/레저_r,H1_패션잡화_r,H2_패션잡화_r,H3_패션잡화_r
고객번호,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0.277311,0.308989,0.333333,0.037815,0.101124,0.060897,0.0,0.000000,0.000000,0.004202,0.002809,0.000000,0.000000,0.02809,0.003205,0.000000,0.000000,0.003205,0.306723,0.300562,0.355769,0.155462,0.132022,0.144231,0.100840,0.095506,0.041667,0.046218,0.000000,0.016026,0.071429,0.030899,0.041667
2,0.141593,0.236162,0.149425,0.091445,0.081181,0.084291,0.0,0.007380,0.000000,0.000000,0.000000,0.000000,0.011799,0.00738,0.011494,0.000000,0.003690,0.015326,0.566372,0.461255,0.379310,0.038348,0.081181,0.076628,0.070796,0.066421,0.203065,0.032448,0.029520,0.026820,0.047198,0.025830,0.053640
3,0.681034,0.672634,0.704545,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.186782,0.173913,0.159091,0.002874,0.000000,0.000000,0.129310,0.153453,0.136364,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.333333,0.319767,0.349810,0.025926,0.069767,0.034221,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.433333,0.331395,0.479087,0.103704,0.104651,0.060837,0.062963,0.110465,0.053232,0.011111,0.029070,0.007605,0.029630,0.034884,0.015209
5,0.320955,0.292308,0.382114,0.074271,0.038462,0.008130,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.002653,0.00000,0.000000,0.002653,0.000000,0.000000,0.551724,0.615385,0.569106,0.013263,0.007692,0.008130,0.015915,0.023077,0.032520,0.002653,0.007692,0.000000,0.015915,0.015385,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19356,0.617021,0.706037,0.654378,0.000000,0.002625,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.340426,0.217848,0.218894,0.000000,0.000000,0.000000,0.042553,0.073491,0.126728,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
19357,0.583333,0.710588,0.684327,0.000000,0.002353,0.008830,0.0,0.016471,0.024283,0.000000,0.016471,0.024283,0.000000,0.00000,0.000000,0.000000,0.002353,0.002208,0.166667,0.134118,0.174393,0.250000,0.025882,0.011038,0.000000,0.075294,0.059603,0.000000,0.009412,0.002208,0.000000,0.007059,0.008830
19359,0.647059,0.713483,0.779126,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.294118,0.232210,0.179612,0.000000,0.000000,0.000000,0.058824,0.054307,0.041262,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [126]:
with open('/content/drive/MyDrive/공유문서/1차 프로젝트/5조/진짜진짜최종/123(train)/분류_r_123.pkl','wb') as train_save:
  pickle.dump(train,train_save)

In [102]:
def test_df(a,b,c,d):
    A = pd.concat([a,b],axis=1)
    train = pd.concat([A,c],axis=1)
    test = pd.concat([train,d],axis=1)
    return test

test = test_df(H1,H2,H3,H4)

In [103]:
test

,H1_가공식품_r,H1_가구/인테리어_r,H1_교육/문화용품_r,H1_기타_r,H1_디지털/가전_r,H1_명품_r,H1_신선식품_r,H1_의류_r,H1_일상용품_r,H1_전문스포츠/레저_r,H1_패션잡화_r,H2_가공식품_r,H2_가구/인테리어_r,H2_교육/문화용품_r,H2_기타_r,H2_디지털/가전_r,H2_명품_r,H2_신선식품_r,H2_의류_r,H2_일상용품_r,H2_전문스포츠/레저_r,H2_패션잡화_r,H3_가공식품_r,H3_가구/인테리어_r,H3_교육/문화용품_r,H3_기타_r,H3_디지털/가전_r,H3_명품_r,H3_신선식품_r,H3_의류_r,H3_일상용품_r,H3_전문스포츠/레저_r,H3_패션잡화_r,H4_가공식품_r,H4_가구/인테리어_r,H4_교육/문화용품_r,H4_기타_r,H4_디지털/가전_r,H4_명품_r,H4_신선식품_r,H4_의류_r,H4_일상용품_r,H4_전문스포츠/레저_r,H4_패션잡화_r
고객번호,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0.277311,0.037815,0.0,0.004202,0.000000,0.000000,0.306723,0.155462,0.100840,0.046218,0.071429,0.308989,0.101124,0.000000,0.002809,0.02809,0.000000,0.300562,0.132022,0.095506,0.000000,0.030899,0.333333,0.060897,0.000000,0.000000,0.003205,0.003205,0.355769,0.144231,0.041667,0.016026,0.041667,0.278146,0.033113,0.00000,0.033113,0.013245,0.0,0.231788,0.304636,0.033113,0.019868,0.052980
2,0.141593,0.091445,0.0,0.000000,0.011799,0.000000,0.566372,0.038348,0.070796,0.032448,0.047198,0.236162,0.081181,0.007380,0.000000,0.00738,0.003690,0.461255,0.081181,0.066421,0.029520,0.025830,0.149425,0.084291,0.000000,0.000000,0.011494,0.015326,0.379310,0.076628,0.203065,0.026820,0.053640,0.220408,0.126531,0.00000,0.000000,0.008163,0.0,0.371429,0.122449,0.089796,0.012245,0.048980
3,0.681034,0.000000,0.0,0.000000,0.000000,0.000000,0.186782,0.002874,0.129310,0.000000,0.000000,0.672634,0.000000,0.000000,0.000000,0.00000,0.000000,0.173913,0.000000,0.153453,0.000000,0.000000,0.704545,0.000000,0.000000,0.000000,0.000000,0.000000,0.159091,0.000000,0.136364,0.000000,0.000000,0.620690,0.000000,0.00000,0.000000,0.000000,0.0,0.262931,0.000000,0.116379,0.000000,0.000000
4,0.333333,0.025926,0.0,0.000000,0.000000,0.000000,0.433333,0.103704,0.062963,0.011111,0.029630,0.319767,0.069767,0.000000,0.000000,0.00000,0.000000,0.331395,0.104651,0.110465,0.029070,0.034884,0.349810,0.034221,0.000000,0.000000,0.000000,0.000000,0.479087,0.060837,0.053232,0.007605,0.015209,0.471111,0.053333,0.00000,0.000000,0.000000,0.0,0.324444,0.080000,0.044444,0.008889,0.017778
5,0.320955,0.074271,0.0,0.000000,0.002653,0.002653,0.551724,0.013263,0.015915,0.002653,0.015915,0.292308,0.038462,0.000000,0.000000,0.00000,0.000000,0.615385,0.007692,0.023077,0.007692,0.015385,0.382114,0.008130,0.000000,0.000000,0.000000,0.000000,0.569106,0.008130,0.032520,0.000000,0.000000,0.382022,0.101124,0.00000,0.000000,0.000000,0.0,0.471910,0.022472,0.011236,0.000000,0.011236
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19356,0.617021,0.000000,0.0,0.000000,0.000000,0.000000,0.340426,0.000000,0.042553,0.000000,0.000000,0.706037,0.002625,0.000000,0.000000,0.00000,0.000000,0.217848,0.000000,0.073491,0.000000,0.000000,0.654378,0.000000,0.000000,0.000000,0.000000,0.000000,0.218894,0.000000,0.126728,0.000000,0.000000,0.617788,0.000000,0.00000,0.000000,0.000000,0.0,0.278846,0.004808,0.096154,0.002404,0.000000
19357,0.583333,0.000000,0.0,0.000000,0.000000,0.000000,0.166667,0.250000,0.000000,0.000000,0.000000,0.710588,0.002353,0.016471,0.016471,0.00000,0.002353,0.134118,0.025882,0.075294,0.009412,0.007059,0.684327,0.008830,0.024283,0.024283,0.000000,0.002208,0.174393,0.011038,0.059603,0.002208,0.008830,0.672515,0.002924,0.01462,0.023392,0.000000,0.0,0.192982,0.011696,0.070175,0.005848,0.005848
19359,0.647059,0.000000,0.0,0.000000,0.000000,0.000000,0.294118,0.000000,0.058824,0.000000,0.000000,0.713483,0.000000,0.000000,0.000000,0.00000,0.000000,0.232210,0.000000,0.054307,0.000000,0.000000,0.779126,0.000000,0.000000,0.000000,0.000000,0.000000,0.179612,0.000000,0.041262,0.000000,0.000000,0.840108,0.000000,0.00000,0.000000,0.000000,0.0,0.124661,0.000000,0.035230,0.000000,0.000000


In [117]:
testlist = ['H1','H2','H3','H4']
c_li = list(H1_r.columns)

testList = []
for col in c_li:
  for h in testlist:
    testList.append(f'{h}_{col}')
test = test[testList]

test

,H1_가공식품_r,H2_가공식품_r,H3_가공식품_r,H4_가공식품_r,H1_가구/인테리어_r,H2_가구/인테리어_r,H3_가구/인테리어_r,H4_가구/인테리어_r,H1_교육/문화용품_r,H2_교육/문화용품_r,H3_교육/문화용품_r,H4_교육/문화용품_r,H1_기타_r,H2_기타_r,H3_기타_r,H4_기타_r,H1_디지털/가전_r,H2_디지털/가전_r,H3_디지털/가전_r,H4_디지털/가전_r,H1_명품_r,H2_명품_r,H3_명품_r,H4_명품_r,H1_신선식품_r,H2_신선식품_r,H3_신선식품_r,H4_신선식품_r,H1_의류_r,H2_의류_r,H3_의류_r,H4_의류_r,H1_일상용품_r,H2_일상용품_r,H3_일상용품_r,H4_일상용품_r,H1_전문스포츠/레저_r,H2_전문스포츠/레저_r,H3_전문스포츠/레저_r,H4_전문스포츠/레저_r,H1_패션잡화_r,H2_패션잡화_r,H3_패션잡화_r,H4_패션잡화_r
고객번호,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0.277311,0.308989,0.333333,0.278146,0.037815,0.101124,0.060897,0.033113,0.0,0.000000,0.000000,0.00000,0.004202,0.002809,0.000000,0.033113,0.000000,0.02809,0.003205,0.013245,0.000000,0.000000,0.003205,0.0,0.306723,0.300562,0.355769,0.231788,0.155462,0.132022,0.144231,0.304636,0.100840,0.095506,0.041667,0.033113,0.046218,0.000000,0.016026,0.019868,0.071429,0.030899,0.041667,0.052980
2,0.141593,0.236162,0.149425,0.220408,0.091445,0.081181,0.084291,0.126531,0.0,0.007380,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.011799,0.00738,0.011494,0.008163,0.000000,0.003690,0.015326,0.0,0.566372,0.461255,0.379310,0.371429,0.038348,0.081181,0.076628,0.122449,0.070796,0.066421,0.203065,0.089796,0.032448,0.029520,0.026820,0.012245,0.047198,0.025830,0.053640,0.048980
3,0.681034,0.672634,0.704545,0.620690,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.186782,0.173913,0.159091,0.262931,0.002874,0.000000,0.000000,0.000000,0.129310,0.153453,0.136364,0.116379,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.333333,0.319767,0.349810,0.471111,0.025926,0.069767,0.034221,0.053333,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.433333,0.331395,0.479087,0.324444,0.103704,0.104651,0.060837,0.080000,0.062963,0.110465,0.053232,0.044444,0.011111,0.029070,0.007605,0.008889,0.029630,0.034884,0.015209,0.017778
5,0.320955,0.292308,0.382114,0.382022,0.074271,0.038462,0.008130,0.101124,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.002653,0.00000,0.000000,0.000000,0.002653,0.000000,0.000000,0.0,0.551724,0.615385,0.569106,0.471910,0.013263,0.007692,0.008130,0.022472,0.015915,0.023077,0.032520,0.011236,0.002653,0.007692,0.000000,0.000000,0.015915,0.015385,0.000000,0.011236
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19356,0.617021,0.706037,0.654378,0.617788,0.000000,0.002625,0.000000,0.000000,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.340426,0.217848,0.218894,0.278846,0.000000,0.000000,0.000000,0.004808,0.042553,0.073491,0.126728,0.096154,0.000000,0.000000,0.000000,0.002404,0.000000,0.000000,0.000000,0.000000
19357,0.583333,0.710588,0.684327,0.672515,0.000000,0.002353,0.008830,0.002924,0.0,0.016471,0.024283,0.01462,0.000000,0.016471,0.024283,0.023392,0.000000,0.00000,0.000000,0.000000,0.000000,0.002353,0.002208,0.0,0.166667,0.134118,0.174393,0.192982,0.250000,0.025882,0.011038,0.011696,0.000000,0.075294,0.059603,0.070175,0.000000,0.009412,0.002208,0.005848,0.000000,0.007059,0.008830,0.005848
19359,0.647059,0.713483,0.779126,0.840108,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.294118,0.232210,0.179612,0.124661,0.000000,0.000000,0.000000,0.000000,0.058824,0.054307,0.041262,0.035230,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [128]:
with open('/content/drive/MyDrive/공유문서/1차 프로젝트/5조/진짜진짜최종/4(test)/분류_r_1234.pkl','wb') as test_save:
  pickle.dump(test,test_save)